<a href="https://colab.research.google.com/github/chj1330/realtimeMultiSpeakerMultiEmotionTTS/blob/main/DNN%20%EC%9D%8C%EC%84%B1%20%ED%95%A9%EC%84%B1%EA%B8%B0%20DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실시간 다화자 다감정 음성 합성 시스템 시연


Multi-speaker Emotional FastSpeech + ParallelWaveGAN 

Reference
- ESPnet: https://github.com/espnet/espnet
- ParallelWaveGAN: https://github.com/kan-bayashi/ParallelWaveGAN



## Install & Setup (3~4분 소요)

In [ ]:
# install minimal components
!pip install -q parallel_wavegan PyYaml unidecode ConfigArgparse torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!git clone -q https://github.com/kan-bayashi/espnet.git -b fix_import
!cd espnet && git fetch && git checkout -b v.0.7.0 4ad3247c850bb6696e4e2c3f7633c0153463dded

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 245kB 14.9MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 703.8MB 25kB/s 
     |████████████████████████████████| 6.6MB 37kB/s 
     |████████████████████████████████| 184kB 17.3MB/s 
     |████████████████████████████████| 317kB 20.3MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
Switched to a new branch 'v.0.7.0'


### Download pretrained feature generation model

Multi-speaker Emotional FastSpeech

In [ ]:
# download pretrained model
import os
if not os.path.exists("downloads/ptFastSpeech"):
    !./espnet/utils/download_from_google_drive.sh \
       https://drive.google.com/open?id=14JKGJ5lfM6ATaavDAlJyCGA81BAden9x downloads/ptFastSpeech tar.gz

# set path
trans_type = "char"
dict_path = "downloads/ptFastSpeech/data/lang_1char/EMO_char_train_no_dev_units.txt"
model_path = "downloads/ptFastSpeech/exp/EMO_char_train_no_dev_pytorch_train_fastspeech.sgst2.spkid/results/model.loss.best"
cmvn_path = "downloads/ptFastSpeech/data/EMO_char_train_no_dev/cmvn.ark"

print("Sucessfully finished download.")

--2020-10-12 08:45:41--  https://drive.google.com/uc?export=download&id=14JKGJ5lfM6ATaavDAlJyCGA81BAden9x
Resolving drive.google.com (drive.google.com)... 108.177.126.100, 108.177.126.113, 108.177.126.139, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘downloads/ptFastSpeech/CYPQRi.tar.gz’

downloads/ptFastSpe     [ <=>                ]   3.18K  --.-KB/s    in 0s      

2020-10-12 08:45:41 (36.8 MB/s) - ‘downloads/ptFastSpeech/CYPQRi.tar.gz’ saved [3256]


gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3256    0  3256    0     0  14800      0 --:--:-- --:--:-- --:--:-- 14733
  % Total    % Received % Xferd  Average Speed   Time    Time    

### Download pretrained vocoder model

Parallel WaveGAN

In [ ]:
# download pretrained model
import os
if not os.path.exists("downloads/ptParallelWavegan"):
    !./espnet/utils/download_from_google_drive.sh \
        https://drive.google.com/open?id=1GVVG9lw6Aq2a-C6au7KCMtp186GtsXrk downloads/ptParallelWavegan tar.gz

# set path
vocoder_path = "downloads/ptParallelWavegan/checkpoint-255000steps.pkl"
vocoder_conf = "downloads/ptParallelWavegan/config.yml"
vocoder_stat =  "downloads/ptParallelWavegan/stats.h5"

print("Sucessfully finished download.")

# add path
import sys
sys.path.append("espnet")
sys.path.append("downloads/ptFastSpeech")

--2020-10-12 08:45:53--  https://drive.google.com/uc?export=download&id=1GVVG9lw6Aq2a-C6au7KCMtp186GtsXrk
Resolving drive.google.com (drive.google.com)... 108.177.119.101, 108.177.119.102, 108.177.119.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-9s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d5ua3kau0v2bvaa5k2u0sskk8a9b5n2k/1602492300000/07266571289902820001/*/1GVVG9lw6Aq2a-C6au7KCMtp186GtsXrk?e=download [following]
--2020-10-12 08:45:56--  https://doc-00-9s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d5ua3kau0v2bvaa5k2u0sskk8a9b5n2k/1602492300000/07266571289902820001/*/1GVVG9lw6Aq2a-C6au7KCMtp186GtsXrk?e=download
Resolving doc-00-9s-docs.googleusercontent.com (doc-00-9s-docs.googleusercontent.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connecting to doc-00-9s-docs.googleusercontent.com (d

### Setup

In [ ]:
# define device
import torch
device = torch.device("cuda")

# define E2E-TTS model
from argparse import Namespace
from os.path import join
from espnet.asr.asr_utils import get_model_conf
from espnet.asr.asr_utils import torch_load
from espnet.utils.dynamic_import import dynamic_import
idim, odim, train_args = get_model_conf(model_path)
model_class = dynamic_import(train_args.model_module)
model = model_class(idim, odim, train_args)
torch_load(model_path, model)
model = model.eval().to(device)
inference_args = Namespace(**{
    "threshold": 0.5,"minlenratio": 0.0, "maxlenratio": 10.0,
    # Only for Tacotron 2
    "use_attention_constraint": True, "backward_window": 1,"forward_window":3,
    # Only for fastspeech (lower than 1.0 is faster speech, higher than 1.0 is slower speech)
    "fastspeech_alpha": 1.0,
    })

# define neural vocoder
import yaml
import parallel_wavegan.models
with open(vocoder_conf) as f:
    config = yaml.load(f, Loader=yaml.Loader)
vocoder_class = config.get("generator_type", "ParallelWaveGANGenerator")
vocoder = getattr(parallel_wavegan.models, vocoder_class)(**config["generator_params"])
vocoder.load_state_dict(torch.load(vocoder_path, map_location="cpu")["model"]["generator"])
vocoder.remove_weight_norm()
vocoder = vocoder.eval().to(device)
if config["generator_params"]["out_channels"] > 1:
    from parallel_wavegan.layers import PQMF
    pqmf = PQMF(config["generator_params"]["out_channels"]).to(device)

from espnet.transform.cmvn import CMVN

cmvn = CMVN(cmvn_path, norm_means=True, norm_vars=True, reverse=True)

from sklearn.preprocessing import StandardScaler
import h5py
scaler = StandardScaler()
with h5py.File(vocoder_stat, "r") as f:
    scaler.mean_ = f["mean"][()]
    scaler.scale_ = f['scale'][()]
scaler.n_features_in_ = scaler.mean_.shape[0]
# define text frontend
with open(dict_path) as f:
    lines = f.readlines()
lines = [line.replace("\n", "").split(" ") for line in lines]
char_to_id = {c: int(i) for c, i in lines}

from unidecode import unidecode
import re
class Text2Grp(object):
    def __init__(self, fn_grptable):
        self.grptable = grptable = self.loadTABLE(fn_grptable)
        self.INITIALS = self.grptable[0].split(' ')
        self.G_INITIALS = grptable[1].split(' ')
        self.MEDIALS = grptable[2].split(' ')
        self.G_MEDIALS = grptable[3].split(' ')
        self.FINALS = grptable[4].split(' ')
        self.G_FINALS = grptable[5].split(' ')
        self.SPECIALS = grptable[6].split(' ')
        self.SPECIALS.append(' ')
        self.CHARACTERS = self.INITIALS + self.MEDIALS + self.FINALS + self.SPECIALS
        self.flag_specials = False

        if len(self.INITIALS) != len(self.G_INITIALS):
            print("Error: character_INITIALS and grapheme_INITIALS length mismatch")
            sys.exit(1)
        if len(self.MEDIALS) != len(self.G_MEDIALS):
            print("Error: character_MEDIALS and grapheme_MEDIALS length mismatch")
            sys.exit(1)
        if len(self.FINALS) != len(self.G_FINALS):
            print("Error: character_FINALS and grapheme_FINALS length mismatch")
            sys.exit(1)

    def loadTABLE(self, fn_grptable):
        with open(fn_grptable, 'r', encoding='utf8') as fid:
            grptable = fid.read().split('\n')
        if len(grptable) != 7:
            print("Invalid table format")
            sys.exit(1)
        return grptable

    def check_syllable(self, x):
        return 0xAC00 <= ord(x) <= 0xD7A3


    def split_syllable_char(self, x):
        if len(x) != 1:
            raise ValueError("Input string must have exactly one character.")

        if not self.check_syllable(x):
            raise ValueError(
                "Input string does not contain a valid Korean character.")

        diff = ord(x) - 0xAC00
        _m = diff % 28
        _d = (diff - _m) // 28

        initial_index = _d // 21
        medial_index = _d % 21
        final_index = _m

        if not final_index:
            result_cha = (self.INITIALS[initial_index], self.MEDIALS[medial_index])
            result_grp = (self.G_INITIALS[initial_index], self.G_MEDIALS[medial_index], 'X')
        else:
            result_cha = (
                self.INITIALS[initial_index], self.MEDIALS[medial_index],
                self.FINALS[final_index - 1])
            result_grp = (
                self.G_INITIALS[initial_index], self.G_MEDIALS[medial_index],
                self.G_FINALS[final_index - 1])

        return result_cha, result_grp

    def split_syllables(self, string):

        new_chracter = ""
        new_grapheme = ""
        for c in string:
            if not self.check_syllable(c):
                if (c in self.SPECIALS) or (c == ' '):
                #if (c in self.SPECIALS):
                    new_c = c
                    new_g = c
                else:
                    new_c = ''
                    new_g = ''
                    self.flag_specials = True

            else:
                [c_sent, g_sent] = self.split_syllable_char(c)
                new_c = "".join(c_sent)
                new_g = "".join(g_sent)
            new_chracter += new_c
            new_grapheme += new_g

        return new_chracter, new_grapheme

def custom_english_cleaners(text):
    _whitespace_re = re.compile(r'\s+')
    '''Custom pipeline for English text, including number and abbreviation expansion.'''
    text = unidecode(text)
    text = re.sub(_whitespace_re, ' ', text)
    return text

text2grp = Text2Grp("downloads/ptFastSpeech/data/grp/table.txt")

def frontend(text):
    """Clean text and then convert to id sequence."""
    _, grp = text2grp.split_syllables(text)
    text = custom_english_cleaners(grp)
    #print(f"Cleaned text: {text}")
    charseq = list(text)
    if not charseq[-1] in [',', '.',' !', '?']:
        charseq += '.'
    idseq = []
    for c in charseq:
        if c.isspace():
            idseq += [char_to_id["<space>"]]
        elif c not in char_to_id.keys():
            idseq += [char_to_id["<unk>"]]
        else:
            idseq += [char_to_id[c]]
    idseq += [idim - 1]  # <eos>
    return torch.LongTensor(idseq).view(-1).to(device)

from espnet.utils.cli_readers import file_reader_helper
emotion_scp = 'scp:downloads/ptFastSpeech/exp/EMO_char_train_no_dev_pytorch_train_fastspeech.sgst2.spkid/outputs_model.loss.best_decode_stemb1.0/EMO_char_train_no_dev/emotion.scp'

stemb_dict_f = {}
stemb_dict_m = {}
for idx, (utt_id, stemb1) in enumerate(file_reader_helper(emotion_scp, 'mat'), 1):
    if utt_id[0] == 'f':
        stemb_dict_f[utt_id[2:]] = stemb1
    elif utt_id[0] == 'm':
        stemb_dict_m[utt_id[2:]] = stemb1

female_spk_list = ['0', '1', '2', '3', '4', '10' , '11', '12', '13', '14']
male_spk_list = ['5', '6', '7', '8', '9', '15', '16', '17', '18', '19']
spemb_list = female_spk_list + male_spk_list



print("Now ready to synthesize!")

Now ready to synthesize!


## Synthesis

In [ ]:
import time
print("문장을 입력해주세요.")
input_text = input()
print("화자 번호를 입력해주세요. *Option* 여자: 0~4, 10~14 / 남자: 5~9, 15~19")
input_speaker = input()
print("감정을 입력해주세요. *Option* neutral, happy, sad, angry")
input_emotion = input()
print("감정 세기를 입력해주세요. *Option* 0.5(약하게), 1.0(적당하게), 2.0(세게)")
input_weight = input()

pad_fn = torch.nn.ReplicationPad1d(
    config["generator_params"].get("aux_context_window", 0))
use_noise_input = vocoder_class == "ParallelWaveGANGenerator"
with torch.no_grad():
    start = time.time()
    if input_speaker in female_spk_list:
      stemb_dict_in = stemb_dict_f
    elif input_speaker in male_spk_list:
      stemb_dict_in = stemb_dict_m
    spemb = torch.LongTensor([int(input_speaker)]).view(-1).to(device)
    stemb = torch.FloatTensor(stemb_dict_in[input_emotion]).view(-1).to(device) * float(input_weight)
    x = frontend(input_text)
    c, _, _ = model.inference(x, None, inference_args, spemb=spemb, stemb=stemb)
    xx_denorm = cmvn(c.cpu().numpy())
    c = torch.FloatTensor(scaler.transform(xx_denorm))
    c = pad_fn(c.unsqueeze(0).transpose(2, 1)).to(device)
    xx = (c,)
    if use_noise_input:
        z_size = (1, 1, (c.size(2) - sum(pad_fn.padding)) * config["hop_size"])
        z = torch.randn(z_size).to(device)
        xx = (z,) + xx
    if config["generator_params"]["out_channels"] == 1:
        y = vocoder(*xx).view(-1)
    else:
        y = pqmf.synthesis(vocoder(*xx)).view(-1)    
rtf = (time.time() - start) / (len(y) / config["sampling_rate"])
print(f"RTF = {rtf:5f}")

from IPython.display import display, Audio
display(Audio(y.view(-1).cpu().numpy(), rate=config["sampling_rate"]))

문장을 입력해주세요.
안녕하세요.
화자 번호를 입력해주세요. *Option* 여자: 0~4, 10~14 / 남자: 5~9, 15~19
0
감정을 입력해주세요. *Option* neutral, happy, sad, angry
neutral
감정 세기를 입력해주세요. *Option* 0.5(약하게), 1.0(적당하게), 2.0(세게)
